### End to End demo of starting spot instance in AWS

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from aws_setup import *

#### Define parameters

In [3]:
name='fast-ai'

#### Get Existing VPC by tag name

In [4]:
vpc = get_vpc(name); vpc

ec2.Vpc(id='vpc-6e6b2a17')

#### Request Spot instance

In [8]:
instance_name = f'{name}'
instance_type = 't2.micro'

In [9]:
spot_prices = get_spot_prices(); spot_prices[instance_type]

'0.008100'

In [10]:
instance = create_spot_instance(instance_name, vpc, spot_price='0.5', instance_type=instance_type); instance

Waiting on spot fullfillment...
Fullfillment completed. InstanceId: i-0c5f0a03ad0f3347a
Rebooting...
Completed. SSH:  ssh -i ~/.ssh/aws-key-fast-ai.pem ubuntu@34.215.111.243


ec2.Instance(id='i-0c5f0a03ad0f3347a')

Request on demand instance (if spot error)

In [ ]:
# instance = create_instance(instance_name, vpc, instance_type='t2.micro'); instance

In [ ]:
instance = get_instance(instance_name); instance

#### Attach EBS volume (Optional)

In [11]:
volume_tag = 'fast-ai-ebs-volume-test'

In [12]:
_ = attach_volume(instance, volume_tag, device='/dev/xvdf')

Volume attached. Please make sure to ssh into instance to format (if new volume) and mount


### SSH

In [13]:
client = connect_to_instance(instance)

Connecting to SSH...
Exception: [Errno None] Unable to connect to port 22 on 34.215.111.243 Retrying...
Exception: timed out Retrying...
Exception: [Errno None] Unable to connect to port 22 on 34.215.111.243 Retrying...
Connected!


#### Mount EBS

In [14]:
upload_path = Path.cwd()/'upload_scripts/mount_ebs.sh'
upload_file(client, str(upload_path), 'mount_ebs.sh')

In [15]:
out, _ = run_command(client, 'chmod 755 mount_ebs.sh')

run_command returned: 



In [16]:
out, _ = run_command(client, './mount_ebs.sh --device /dev/xvdf') # no reformatting
# out, _ = run_command(client, './mount_ebs.sh --device /dev/xvdf --reformat true')

run_command returned: 



In [17]:
out, _ = run_command(client, 'ls ebs_mount_point') # no reformatting

run_command returned: 
saved.txt



#### Mount EFS

efs_addr = get_efs_address('fast-ai-efs'); efs_addr

In [19]:
out, _ = run_command(client, 'mkdir ~/efs_mount_point')

run_command returned: 



In [20]:
efs_mount_cmd = f'sudo mount -t nfs -o nfsvers=4.1,rsize=1048576,wsize=1048576,hard,timeo=600,retrans=2 {efs_addr}:/ ~/efs_mount_point'

In [21]:
out, _ = run_command(client, efs_mount_cmd)

run_command returned: 



In [22]:
out, _ = run_command(client, 'ls efs_mount_point') # no reformatting

run_command returned: 
efs_saved.txt



### TODO

* tmux
* Auto shutdown after training
* Convert to script